In [103]:
import pandas as pd

In [104]:
df1 = pd.read_csv(r'E:\ML Learning\PROJECT_SENTIMENT_ANALYSIS_WHITE_HOUSE\STEP_4_CORRELATION\data_files\roberta_scored_briefings.csv')
df2 = pd.read_csv(r'E:\ML Learning\PROJECT_SENTIMENT_ANALYSIS_WHITE_HOUSE\STEP_4_CORRELATION\data_files\approvalRating2.csv')

In [105]:
#CHANGE DF2 TO DATE TIME
df2['end_date'] = pd.to_datetime(df2['end_date'])

In [106]:
#w will analyze using the year-month as the approval rating is done monthly
df_roberta = df1.drop(['quarter', 'url'], axis = 1)
#drop duplicates / basic cleaning.
df_roberta = df_roberta.drop_duplicates()
df_roberta = df_roberta.dropna()
df_roberta

,content,roberta_neg,roberta_neu,roberta_pos,briefing_type,date,title
0,"President Joseph R. Biden, Jr. spoke today wit...",0.002994,0.155596,0.841410,Statements and Releases,2024-07-02,Readout of President Joe Biden’s Call with Pre...
1,1:50 P.M. EDTMS. JEAN-PIERRE: Hi. Good afterno...,0.023508,0.531409,0.445083,Press Briefings,2024-07-03,Press Briefing by Press Secretary Karine Jean-...
2,President Biden spoke this morning with Prime ...,0.005338,0.228428,0.766233,Statements and Releases,2024-07-04,Readout of President Joe Biden’s Call with Pri...
3,"The South LawnReverend Lee, the President and ...",0.019021,0.145596,0.835383,Speeches and Remarks,2024-07-04,Remarks as Prepared for Delivery by First Lady...
4,"With today’s report that 206,000 jobs were cre...",0.151755,0.456691,0.391554,Statements and Releases,2024-07-05,Statement from President Joe Biden on the June...
...,...,...,...,...,...,...,...
10735,President Biden and Vice President Harris are ...,0.010377,0.714800,0.274824,Statements and Releases,2024-06-26,Biden-⁠Harris Administration Announces the App...
10736,"– Today, Vice President Harris announced the r...",0.065244,0.560964,0.373792,Statements and Releases,2024-06-26,FACT SHEET: Vice President Harris Announces Fi...
10737,Our Nation has made tremendous progress in adv...,0.112505,0.658855,0.228641,Presidential Actions,2024-06-26,A Proclamation on Granting Pardon for Certain ...
10738,Every person in America should be able to acce...,0.019261,0.209876,0.770863,Statements and Releases,2024-06-26,Statement from Vice President Kamala Harris on...


In [107]:
# making the roberta scoring monthly
df_roberta['year_month'] = pd.PeriodIndex(df_roberta['date'] , freq = 'M')
#dropping the unused : content, title, date
df_roberta =df_roberta.drop(['content' , 'date', 'title', 'briefing_type'], axis = 1)
df_roberta

,roberta_neg,roberta_neu,roberta_pos,year_month
0,0.002994,0.155596,0.841410,2024-07
1,0.023508,0.531409,0.445083,2024-07
2,0.005338,0.228428,0.766233,2024-07
3,0.019021,0.145596,0.835383,2024-07
4,0.151755,0.456691,0.391554,2024-07
...,...,...,...,...
10735,0.010377,0.714800,0.274824,2024-06
10736,0.065244,0.560964,0.373792,2024-06
10737,0.112505,0.658855,0.228641,2024-06
10738,0.019261,0.209876,0.770863,2024-06


In [108]:
#as the approval poll is performed monthly, we will analyze the data by month.
#add a year-month column to df2
#as we wont use the start date since it starts at the beginning of the month regardless, we will drop it
df2['year_month'] = pd.PeriodIndex(df2['end_date'], freq= 'M')
df_approval = df2.drop(['end_date','start_date' ] , axis = 1)
df_approval

,approving,disapproving,unsure,year_month
0,38,58,4,2024-06
1,39,56,5,2024-05
2,38,58,4,2024-04
3,40,55,5,2024-03
4,38,59,3,2024-02
5,41,54,5,2024-01
6,39,59,2,2023-12
7,37,59,4,2023-11
8,37,59,4,2023-10
9,41,58,1,2023-09


In [109]:
# as we can see, there are only 40 data points representing 40 months ever since Biden took office. Thus, consolidating the scoring data from Roberta to monthly is the choice i go with

#finding the mean for the report 
df_roberta_agg = df_roberta.groupby('year_month' , as_index=False).mean({"roberta_neg"
                                                       ,"roberta_neu"
                                                       ,"roberta_pos"})
df_roberta_agg

,year_month,roberta_neg,roberta_neu,roberta_pos
0,2021-01,0.157980,0.559022,0.282999
1,2021-02,0.129722,0.496442,0.373837
2,2021-03,0.123294,0.479990,0.396716
3,2021-04,0.098924,0.486401,0.414675
4,2021-05,0.110462,0.511549,0.377989
5,2021-06,0.097809,0.462875,0.439315
6,2021-07,0.109535,0.501684,0.388781
7,2021-08,0.092111,0.522088,0.385801
8,2021-09,0.105425,0.484207,0.410368
9,2021-10,0.083667,0.482015,0.434317


In [149]:
#as the census data does not include : 2024-07 and 2022-11, we drop those from roberta
df_roberta_agg_test = df_roberta_agg.set_index('year_month')
df_roberta_agg_test = df_roberta_agg_test.drop(df_roberta_agg_test.loc[['2024-07' , '2022-11']].index)
df_roberta_agg = df_roberta_agg_test

KeyError: "None of ['year_month'] are in the columns"

In [151]:
df_roberta_agg

,roberta_neg,roberta_neu,roberta_pos
year_month,,,
2021-01,0.157980,0.559022,0.282999
2021-02,0.129722,0.496442,0.373837
2021-03,0.123294,0.479990,0.396716
2021-04,0.098924,0.486401,0.414675
2021-05,0.110462,0.511549,0.377989
2021-06,0.097809,0.462875,0.439315
2021-07,0.109535,0.501684,0.388781
2021-08,0.092111,0.522088,0.385801
2021-09,0.105425,0.484207,0.410368


In [180]:
# merging the 2 dataset:
df_approval = df_approval.set_index('year_month').sort_values(by=['year_month'], ascending=True)
df_approval

,approving,disapproving,unsure
year_month,,,
2021-01,57,37,6
2021-02,56,40,4
2021-03,54,42,4
2021-04,57,40,3
2021-05,54,40,6
2021-06,56,42,2
2021-07,50,45,5
2021-08,49,48,3
2021-09,43,55,2


In [181]:
#merging dataframe
merged_df = pd.merge(df_approval, df_roberta_agg, on='year_month', how='outer')

In [182]:
merged_df

,approving,disapproving,unsure,roberta_neg,roberta_neu,roberta_pos
year_month,,,,,,
2021-01,57,37,6,0.157980,0.559022,0.282999
2021-02,56,40,4,0.129722,0.496442,0.373837
2021-03,54,42,4,0.123294,0.479990,0.396716
2021-04,57,40,3,0.098924,0.486401,0.414675
2021-05,54,40,6,0.110462,0.511549,0.377989
2021-06,56,42,2,0.097809,0.462875,0.439315
2021-07,50,45,5,0.109535,0.501684,0.388781
2021-08,49,48,3,0.092111,0.522088,0.385801
2021-09,43,55,2,0.105425,0.484207,0.410368


,approving,disapproving,unsure,year_month
0,38,58,4,2024-06
1,39,56,5,2024-05
2,38,58,4,2024-04
3,40,55,5,2024-03
4,38,59,3,2024-02
5,41,54,5,2024-01
6,39,59,2,2023-12
7,37,59,4,2023-11
8,37,59,4,2023-10
9,41,58,1,2023-09
